## Installations

In [4]:
!pip install -q langchain-huggingface
!pip install -U -q langchain-community
!pip install -q faiss-gpu-cu12
!pip install -U -q duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.

## Import Stuff

In [5]:
# Langchain imports
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.agents import load_tools, initialize_agent, AgentType,Tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers import RegexParser
from langchain.tools import DuckDuckGoSearchRun
from langchain.memory import ConversationBufferMemory

# Other imports
import getpass
import os
from typing import List, Dict, Any, Union
import heapq
from dataclasses import dataclass, field
import random
from graphviz import Digraph
import uuid

## Setup LLM

In [6]:
if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your token: ")

Enter your token: ··········


In [7]:
llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

In [8]:
chat_model = ChatHuggingFace(llm=llm)

## Prompting

In [ ]:
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(
        content="What happens when an unstoppable force meets an immovable object?"
    ),
]

In [ ]:
ai_msg = chat_model.invoke(messages)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(ai_msg.content)

According to common wisdom, when an unstoppable force meets an immovable object, there is a paradoxical situation where logic seems to break down. Both phrases, "unstoppable force" and "immovable object," suggest that one cannot be moved or stopped by any force, making it impossible for both to coexist in the same situation without altering their characteristics. However, the expression is often used as a rhetorical device to emphasize a situation where two seemingly opposing factors come into conflict, resulting in an unpredictable outcome. In reality, such a scenario is highly hypothetical and cannot be observed in nature or in physics. The phrase should hence be interpreted with a grain of salt and is not a scientific principle.


## Agents

### Chain of Thought

In [ ]:
cot_template = """Question: {question}

Let's approach this step by step:
1) First, let's understand what we're being asked
2) Then, break down the problem into smaller parts
3) Finally, solve each part and combine the results

Please show your complete reasoning process:

Reasoning:"""

cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=cot_template
)

# Create a parser to extract the final answer
output_parser = RegexParser(
    regex=r"Therefore, the answer is: (.*)",
    output_keys=["answer"],
    default_output_key="answer"
)

In [ ]:
# Create the main chain of thought class
class ChainOfThoughtReasoner:
    def __init__(self):
        self.llm_chain = LLMChain(
            llm=llm,
            prompt=cot_prompt,
            verbose=True
        )

    def solve(self, question: str) -> dict:
        # Get the full reasoning process
        reasoning = self.llm_chain.run(question)

        try:
            # Try to extract the final answer
            answer = output_parser.parse(reasoning)["answer"]
        except:
            answer = "Could not parse a specific answer from the reasoning"

        return {
            "full_reasoning": reasoning,
            "final_answer": answer
        }

# Example implementation with different types of problems
class ProblemSolver:
    def __init__(self):
        self.reasoner = ChainOfThoughtReasoner()

    def solve_math_problem(self, problem: str) -> dict:
        """Solve mathematical problems with step-by-step reasoning"""
        enhanced_prompt = f"Mathematical Problem: {problem}\n\nShow your work step by step."
        return self.reasoner.solve(enhanced_prompt)

    def solve_word_problem(self, problem: str) -> dict:
        """Solve word problems with logical reasoning"""
        enhanced_prompt = f"Word Problem: {problem}\n\nBreak this down logically."
        return self.reasoner.solve(enhanced_prompt)

    def solve_logic_puzzle(self, puzzle: str, constraints: List[str]) -> dict:
        """Solve logic puzzles considering multiple constraints"""
        constraints_text = "\n".join([f"- {c}" for c in constraints])
        enhanced_prompt = f"""Logic Puzzle: {puzzle}

Given these constraints:
{constraints_text}

Solve this step by step, considering each constraint."""
        return self.reasoner.solve(enhanced_prompt)

In [ ]:
solver = ProblemSolver()

<ipython-input-13-577a9d099bea>:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.llm_chain = LLMChain(


In [ ]:
math_result = solver.solve_math_problem(
        "If a train travels at 60 mph for 2.5 hours, how far does it go?"
    )
print("\nMath Problem Solution:")
print(math_result["full_reasoning"])
print(f"Final Answer: {math_result['final_answer']}")



> Entering new LLMChain chain...
Prompt after formatting:
Question: Mathematical Problem: If a train travels at 60 mph for 2.5 hours, how far does it go?

Show your work step by step.

Let's approach this step by step:
1) First, let's understand what we're being asked
2) Then, break down the problem into smaller parts
3) Finally, solve each part and combine the results

Please show your complete reasoning process:

Reasoning:


<ipython-input-13-577a9d099bea>:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  reasoning = self.llm_chain.run(question)



> Finished chain.

Math Problem Solution:

1) Understand the problem:
   a) Train travels at 60 miles per hour (mph)
   b) It travels for 2.5 hours
   c) We need to find how far it goes

2) Break down the problem:
   a) Distance = Speed * Time
       Distance (in miles) = 60 mph * 2.5 hours
   b) Write the formula using variables
       Distance (in miles) = Speed (in mph) * Time (in hours)
       Distance (in miles) = x (speed in mph) * y (time in hours)

3) Solve each part:
   a) Substitute values in the formula we derived:
       Distance (in miles) = Speed (in mph) * Time (in hours)
       Distance (in miles) = 60 mph * 2.5 hours
       Distance (in miles) = 150 miles

4) Combine the results:
   a) Write the complete answer:
       If a train travels at 60 mph for 2.5 hours, it goes a distance of 150 miles.

So, the complete answer is: "If a train travels at 60 mph for 2.5 hours, it goes a distance of 150 miles."

I hope this helps! Let me know if you have any questions or need an

In [ ]:
word_result = solver.solve_word_problem(
        """Sally has 3 more apples than twice the number of oranges Jane has.
        If Jane has 4 oranges, how many apples does Sally have?"""
    )
print("\nWord Problem Solution:")
print(word_result["full_reasoning"])
print(f"Final Answer: {word_result['final_answer']}")



> Entering new LLMChain chain...
Prompt after formatting:
Question: Word Problem: Sally has 3 more apples than twice the number of oranges Jane has.
        If Jane has 4 oranges, how many apples does Sally have?

Break this down logically.

Let's approach this step by step:
1) First, let's understand what we're being asked
2) Then, break down the problem into smaller parts
3) Finally, solve each part and combine the results

Please show your complete reasoning process:

Reasoning:

> Finished chain.

Word Problem Solution:

1) We are being asked to find how many apples Sally has.
2) To do that, we first need to know how many oranges Jane has, and then find how many apples Sally has based on the relationship between the number of apples Sally has and the number of oranges Jane has.
3) Let's call the number of oranges Jane has x.
4) Since Sally has 3 more apples than twice the number of oranges Jane has, we can set up an equation to represent this relationship:
    Number of apples Sa

In [ ]:
logic_result = solver.solve_logic_puzzle(
        "Determine who owns which pet.",
        [
            "There are three people: Alice, Bob, and Carol",
            "They each own one pet: dog, cat, and bird",
            "Alice doesn't like cats",
            "Bob lives in an apartment that doesn't allow dogs",
            "Carol is allergic to birds"
        ]
    )
print("\nLogic Puzzle Solution:")
print(logic_result["full_reasoning"])
print(f"Final Answer: {logic_result['final_answer']}")



> Entering new LLMChain chain...
Prompt after formatting:
Question: Logic Puzzle: Determine who owns which pet.

Given these constraints:
- There are three people: Alice, Bob, and Carol
- They each own one pet: dog, cat, and bird
- Alice doesn't like cats
- Bob lives in an apartment that doesn't allow dogs
- Carol is allergic to birds

Solve this step by step, considering each constraint.

Let's approach this step by step:
1) First, let's understand what we're being asked
2) Then, break down the problem into smaller parts
3) Finally, solve each part and combine the results

Please show your complete reasoning process:

Reasoning:

> Finished chain.

Logic Puzzle Solution:

1) We are trying to determine who owns which pet based on given constraints
2) To solve this, we need to eliminate options that don't fit the constraints for each person
3) Let's start with Alice, since she doesn't like cats
4) Alice cannot own a cat, so that leaves us with two possible pets for her: dog or bird
5)

### Tree of Thought

In [ ]:
@dataclass
class ThoughtNode:
    state: str
    thought: str
    score: float
    parent: Any = None
    children: List[Any] = field(default_factory=list)
    depth: int = 0
    id: str = field(default_factory=lambda: str(uuid.uuid4()))

    def __lt__(self, other):
        return self.score > other.score

class TreeOfThought:
    def __init__(self, max_depth: int = 8, beam_width: int = 3):
        self.max_depth = max_depth
        self.beam_width = beam_width
        self.root = None

    def evaluate_thought(self, thought: str, current_num: int) -> float:
        target = 21
        num = current_num
        distance_score = 1.0 - (abs(target - num) / target)
        valid_move = any(f"add {n}" in thought.lower() for n in [1, 2])
        valid_bonus = 0.2 if valid_move else 0
        solution_bonus = 0.5 if num == target else 0
        return min(1.0, distance_score + valid_bonus + solution_bonus)

    def generate_thoughts(self, state: str, depth: int) -> List[str]:
        current_num = int(state)
        thoughts = []

        if current_num == 21:
            thoughts.append(f"Reached 21! This is a valid solution.")
            return thoughts

        if current_num > 21:
            thoughts.append(f"Reached {current_num}, which is over 21. Dead end.")
            return thoughts

        thoughts.append(f"Add 1 to {current_num} to get {current_num + 1}")
        thoughts.append(f"Add 2 to {current_num} to get {current_num + 2}")

        return thoughts

    def get_next_state(self, current_state: str, thought: str) -> str:
        import re
        numbers = re.findall(r'\d+', thought)
        return numbers[-1] if numbers else current_state

    def solve(self, initial_state: str) -> List[str]:
        self.root = ThoughtNode(initial_state, "Starting at " + initial_state, 0.5)
        frontier = [self.root]
        best_solution = None
        best_score = 0.0

        for depth in range(self.max_depth):
            next_frontier = []

            for parent in frontier:
                thoughts = self.generate_thoughts(parent.state, depth)

                for thought in thoughts:
                    next_state = self.get_next_state(parent.state, thought)
                    score = self.evaluate_thought(thought, int(next_state))

                    node = ThoughtNode(
                        state=next_state,
                        thought=thought,
                        score=score,
                        parent=parent,
                        depth=depth + 1
                    )
                    parent.children.append(node)
                    next_frontier.append(node)

                    if int(next_state) == 21 and score > best_score:
                        best_solution = node
                        best_score = score

            if best_solution:
                break

            next_frontier.sort(key=lambda x: x.score, reverse=True)
            frontier = next_frontier[:self.beam_width]

            if not frontier:
                break

        if best_solution:
            return self._get_solution_path(best_solution)
        return ["No solution found. Try a different starting number."]

    def _get_solution_path(self, node: ThoughtNode) -> List[str]:
        path = []
        current = node
        while current:
            path.append(current.thought)
            current = current.parent
        return list(reversed(path))

    def visualize_tree(self, highlight_solution=True) -> Digraph:
        """
        Visualize the tree using graphviz.
        If highlight_solution is True, highlights the path to 21 in green.
        """
        if not self.root:
            return None

        # Create a new directed graph
        dot = Digraph(comment='Tree of Thought')
        dot.attr(rankdir='TB')

        # Find solution path nodes if highlighting is requested
        solution_nodes = set()
        if highlight_solution:
            current = None
            # Find a node that reaches 21
            def find_solution_node(node):
                if node.state == '21':
                    return node
                for child in node.children:
                    result = find_solution_node(child)
                    if result:
                        return result
                return None

            solution_node = find_solution_node(self.root)
            if solution_node:
                current = solution_node
                while current:
                    solution_nodes.add(current.id)
                    current = current.parent

        # Function to recursively add nodes and edges
        def add_nodes(node):
            # Create node color based on score and whether it's in solution path
            color = 'lightgreen' if node.id in solution_nodes else \
                   'lightblue' if node.score >= 0.7 else \
                   'lightyellow' if node.score >= 0.4 else 'lightpink'

            # Add node with score and state
            label = f"{node.thought}\nScore: {node.score:.2f}"
            dot.node(node.id, label, style='filled', fillcolor=color)

            # Add edges to children
            for child in node.children:
                dot.edge(node.id, child.id)
                add_nodes(child)

        # Add all nodes and edges starting from root
        add_nodes(self.root)
        return dot

In [ ]:
def solve_and_visualize(start_number: int):
    """
    Solve the puzzle and create a visualization.
    """
    tot = TreeOfThought(max_depth=12, beam_width=3)
    initial_state = str(start_number)

    print(f"Solving puzzle: Reach exactly 21 by adding 1 or 2 to the number")
    print(f"Starting number: {initial_state}")
    print("\nThinking...")

    solution = tot.solve(initial_state)

    print("\nSolution path:")
    for i, step in enumerate(solution, 1):
        print(f"Step {i}: {step}")

    # Create and save visualization
    dot = tot.visualize_tree()
    dot.render('tree_of_thought', view=True, format='png')
    print("\nVisualization has been saved as 'tree_of_thought.png'")

In [ ]:
solve_and_visualize(8)

Solving puzzle: Reach exactly 21 by adding 1 or 2 to the number
Starting number: 8

Thinking...

Solution path:
Step 1: Starting at 8
Step 2: Add 2 to 8 to get 10
Step 3: Add 2 to 10 to get 12
Step 4: Add 2 to 12 to get 14
Step 5: Add 2 to 14 to get 16
Step 6: Add 1 to 16 to get 17
Step 7: Add 2 to 17 to get 19
Step 8: Add 2 to 19 to get 21

Visualization has been saved as 'tree_of_thought.png'


### ReAct

In [9]:
class ReActAgent:
    def __init__(self, llm):
        # Use the provided LLM
        self.llm = llm

        # Initialize the memory
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )

        # Initialize tools
        self.tools = self._initialize_tools()

        # Initialize the agent
        self.agent = initialize_agent(
            tools=self.tools,
            llm=self.llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            memory=self.memory,
            verbose=True,
            max_iterations=5,
            early_stopping_method="generate",
            handle_parsing_errors=True
        )

    def _initialize_tools(self) -> List[Tool]:
        """Initialize the tools available to the agent"""

        # Search tool
        search = DuckDuckGoSearchRun()

        # Calculator tool for math operations
        def calculator(expression: str) -> str:
            try:
                return str(eval(expression))
            except Exception as e:
                return f"Error evaluating expression: {str(e)}"

        # Weather tool (mock implementation)
        def get_weather(location: str) -> str:
            return f"Mock weather data for {location}: 72°F, Sunny"

        # Define the tools
        tools = [
            Tool(
                name="Search",
                func=search.run,
                description="Useful for searching the internet to find information about topics"
            ),
            Tool(
                name="Calculator",
                func=calculator,
                description="Useful for performing mathematical calculations"
            ),
            Tool(
                name="Weather",
                func=get_weather,
                description="Get the current weather for a location"
            )
        ]

        return tools

    def add_tool(self, tool: Tool) -> None:
        """Add a new tool to the agent"""
        self.tools.append(tool)
        self.agent = initialize_agent(
            tools=self.tools,
            llm=self.llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            memory=self.memory,
            verbose=True
        )

    def run(self, query: str) -> str:
        """Run the agent on a query"""
        try:
            response = self.agent.run(query)
            return response
        except Exception as e:
            return f"Error running agent: {str(e)}"

    def get_memory(self) -> List[str]:
        """Get the conversation history"""
        return self.memory.chat_memory.messages

class CustomTool:
    """Helper class to create custom tools"""
    @staticmethod
    def create(name: str, func: callable, description: str) -> Tool:
        return Tool(
            name=name,
            func=func,
            description=description
        )


In [10]:
def main():
    # Initialize the ReAct agent with your LLM
    agent = ReActAgent(llm=llm)  # Using your existing llm

    # Add a custom tool
    def word_count(text: str) -> str:
        return f"Word count: {len(text.split())}"

    custom_tool = CustomTool.create(
        name="WordCounter",
        func=word_count,
        description="Counts the number of words in a text"
    )
    agent.add_tool(custom_tool)

    # Example queries
    queries = [
        "What's the weather in New York?",
        "Calculate 234 * 456",
        "Search for the latest news about artificial intelligence",
        "Count the words in: The quick brown fox jumps over the lazy dog"
    ]

    # Run queries
    for query in queries:
        print(f"\nQuery: {query}")
        response = agent.run(query)
        print(f"Response: {response}")

In [11]:
main()

<ipython-input-9-f801dc50d223>:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(
<ipython-input-9-f801dc50d223>:16: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  self.agent = initialize_agent(
<ipython-input-9-f801dc50d223>:79: LangChainDeprecationWarning: The method `Chain.run` was 


Query: What's the weather in New York?


> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


 Let's use the Weather tool to get the current weather for New York.
Action: Weather
Action Input: New York
Observation: Mock weather data for New York: 72°F, Sunny
Thought: Now I know the weather in New York.
Final Answer: The weather in New York is currently 72°F and sunny.

> Finished chain.
Response: The weather in New York is currently 72°F and sunny.

Query: Calculate 234 * 456


> Entering new AgentExecutor chain...
 Let's use the Calculator tool to perform this simple arithmetic operation
Action: Calculator
Action Input: 234*456
Observation: 106704
Thought: Great, I got the correct answer!
Final Answer: The product of 234 and 456 is 106704.

> Finished chain.
Response: The product of 234 and 456 is 106704.

Query: Search for the latest news about artificial intelligence


> Entering new AgentExecutor chain...
 I need to find reliable sources and exclude any sensationalized articles
Action: Search
Action Input: "latest news artificial intelligence" -2019 -sensational
Observation